In [ ]:
%load_ext autoreload
%autoreload 2

import numpy
import torch
import torch.autograd
from pathlib import Path
from tifffile import imread, imsave
from scipy.ndimage import affine_transform


import matplotlib.pyplot as plt

from lnet.config.dataset.registration import Heart_tightCrop_Transform, BDVTransform

# K:\LF_computed\LenseLeNet_Microscope\20191208_dynamic_static_heart\fish2\static\staticHeartFOV\completeSlideThrough_125steps_stepsize2\2019-12-09_09.52.38\stack_4_channel_1\TP_00000\LC

In [ ]:
# path = "/g/kreshuk/LF_computed/LenseLeNet_Microscope/20191208_dynamic_static_heart/fish2/static/staticHeartFOV/completeSlideThrough_125steps_stepsize2/2019-12-09_09.52.38/stack_4_channel_1/TP_00000/LC/Cam_Left_registered_copy.tif"


In [ ]:
# img = imread(path)

In [ ]:
# out_path = "/g/kreshuk/LF_computed/LenseLeNet_Microscope/20191208_dynamic_static_heart/fish2/static/staticHeartFOV/completeSlideThrough_125steps_stepsize2/2019-12-09_09.52.38/stack_4_channel_1/TP_00000/LC/Cam_Left_registered_compressed.tif"

In [ ]:
# imsave(out_path, img, compress=6, bigtiff=True)

In [ ]:
# img_comp = imread(out_path)

In [ ]:
# for i in range(1, 5, 2):
#     out_path = f"/g/kreshuk/LF_computed/LenseLeNet_Microscope/20191208_dynamic_static_heart/fish2/static/staticHeartFOV/completeSlideThrough_125steps_stepsize2/2019-12-09_09.52.38/stack_4_channel_1/TP_00000/LC/Cam_Left_registered_compressed_{i}.tif"
#     imsave(out_path, img, compress=i, bigtiff=True)

In [ ]:
ipt_path = "/g/kreshuk/LF_partially_restored/LenseLeNet_Microscope/20191208_dynamic_static_heart/fish1/static/Heart_tightCrop/centered_5steps_stepsize8/2019-12-09_02.16.30/stack_4_channel_1/TP_00000/LC/Cam_Left.tif"
# ipt_path = "/g/kreshuk/LF_partially_restored/LenseLeNet_Microscope/20191208_dynamic_static_heart/beads/after_fish2/definitelyNotMoving/Heart_tightCrop/200msExp/2019-12-09_22.23.27/Cam_Left.tif"
# ipt_path = "/g/kreshuk/LF_partially_restored/LenseLeNet_Microscope/20191208_dynamic_static_heart/beads/after_fish2/definitelyNotMoving/Heart_tightCrop/200msExp/2019-12-09_22.23.27/stack_1_channel_1/TP_00000/LC/Cam_Left.tif"

expected_path = "/g/kreshuk/LF_partially_restored/LenseLeNet_Microscope/20191208_dynamic_static_heart/fish1/static/Heart_tightCrop/centered_5steps_stepsize8/2019-12-09_02.16.30/stack_4_channel_1/TP_00000/LC/Cam_Left_registered.tif"
# expected_path = "/g/kreshuk/LF_partially_restored/LenseLeNet_Microscope/20191208_dynamic_static_heart/beads/after_fish2/definitelyNotMoving/Heart_tightCrop/200msExp/2019-12-09_22.23.27/Cam_Left_registered.tif"
# expected_path = "/g/kreshuk/LF_partially_restored/LenseLeNet_Microscope/20191208_dynamic_static_heart/beads/after_fish2/definitelyNotMoving/Heart_tightCrop/200msExp/2019-12-09_22.23.27/stack_1_channel_1/TP_00000/LC/Cam_Left_registered.tif"

ipt = imread(ipt_path)
expected = imread(expected_path)
tmp = (Path() / "tmp/").resolve()

print(ipt.shape)
print(expected.shape)

In [ ]:


plt.imshow(ipt.max(axis=0))


In [ ]:
plt.imshow(expected.max(axis=0))

In [ ]:
print(expected.shape, expected.min(), expected.max(), expected.mean())

In [ ]:
print(ipt.shape)
print(expected.shape)
out_shape = numpy.array(expected.shape)

# out = Heart_tightCrop_Transform.apply(ipt.astype("float"))  # (241, 1451, 1651) 0.0 18805.581943353336 160.23466656346213

# trf = Heart_tightCrop_Transform(output_shape=out_shape, order=1)
trf = Heart_tightCrop_Transform(output_shape=tuple(out_shape), order=0)
# def trf(img):
#     affine_matrix = BDVTransform.concat_affine_matrices(
#             [BDVTransform.bdv_trafo_to_affine_matrix(at) for at in Heart_tightCrop_Transform.affine_transforms]
#         )
#     return affine_transform(img, numpy.linalg.inv(affine_matrix), output_shape=out_shape, order=1)
    
# out = trf.apply(ipt.astype("float"))
out = trf.forward(torch.from_numpy(ipt.astype("float")[None, None, ...])).numpy()[0, 0]
print(out.shape, out.min(), out.max(), out.mean())

In [ ]:

# needs_grad = torch.ones((1,), requires_grad=True)
gpu_needs_grad = torch.ones((1,), requires_grad=True, device=torch.device("cuda:0"))
needs_grad = gpu_needs_grad.to(torch.device("cpu"))
# mini.cuda()
# needs_grad = needs_grad.cuda()
# needs_grad.cuda()
# needs_grad2 = torch.ones((1,), requires_grad=True)
# small_ipt = torch.from_numpy(ipt[100:149, :20, :20].astype("float")[None, None, ...]).cuda()
# grad_out = (small_ipt + needs_grad).sum()#.to(torch.device("cpu")) + needs_grad2

grad_out = needs_grad * 2
# grad_out = needs_grad * 2
print("out on cuda", grad_out.is_cuda)

In [ ]:
print(needs_grad.requires_grad, needs_grad.grad)
print(gpu_needs_grad.requires_grad, gpu_needs_grad.grad)
# print(needs_grad2.grad)
grad_out.backward()
print(needs_grad.requires_grad, needs_grad.grad)
print(gpu_needs_grad.requires_grad, gpu_needs_grad.grad)
# print(needs_grad2.grad)

In [ ]:
# print(needs_grad.requires_grad, needs_grad.is_cuda)
# print(needs_grad.to(torch.device("cpu")).requires_grad)
# print(needs_grad.requires_grad, needs_grad.is_cuda)

In [ ]:

small_ipt = ipt[100:149, :20, :20]
print(small_ipt.shape)
out_shape = numpy.array([s for s in small_ipt.shape])

# out = Heart_tightCrop_Transform.apply(ipt.astype("float"))  # (241, 1451, 1651) 0.0 18805.581943353336 160.23466656346213

# trf = Heart_tightCrop_Transform(output_shape=out_shape, order=1)
trf = Heart_tightCrop_Transform(output_shape=tuple(out_shape), order=0)
# trf.cuda()
# def trf(img):
#     affine_matrix = BDVTransform.concat_affine_matrices(
#             [BDVTransform.bdv_trafo_to_affine_matrix(at) for at in Heart_tightCrop_Transform.affine_transforms]
#         )
#     return affine_transform(img, numpy.linalg.inv(affine_matrix), output_shape=out_shape, order=1)
    
# out = trf.apply(ipt.astype("float"))
small_torch_ipt = torch.from_numpy(ipt.astype("float")[None, None, ...])
small_cuda_ipt = small_torch_ipt.cuda()
# small_out = trf.apply(small_torch_ipt[0, 0].numpy())
# small_out = trf.forward(small_torch_ipt).numpy()[0, 0]
small_out = trf.forward(small_cuda_ipt).cpu().numpy()[0, 0]
print(small_out.shape, small_out.min(), small_out.max(), small_out.mean())

In [ ]:
plt.imshow(small_out.max(axis=0))
plt.colorbar()

In [ ]:
# torch comparison
plt.imshow(out.max(axis=0))
plt.colorbar()

In [ ]:
plt.imshow(out.max(axis=0) - expected.max(axis=0))
plt.colorbar()

In [ ]:
# failed torch comparison
plt.imshow(out.max(axis=0))
plt.colorbar()

In [ ]:
plt.imshow(out.max(axis=0) - expected.max(axis=0))
plt.colorbar()

In [ ]:
# (838, 1273, 1463) 0.0 15518.998620498136 62.415287760082315
# scipy comparison
plt.imshow(out.max(axis=0))
plt.colorbar()

In [ ]:
plt.imshow(out.max(axis=0) - expected.max(axis=0))
plt.colorbar()

In [ ]:
print('ok')

In [ ]:
python transformed (max proj) and diff of pyton trf and tischi-trf
# floatout = out.astype("float")
# floatexpected = expected.astype("float")
# floatdiff = floatout - floatexpected
# 
# print(floatdiff.max())

In [ ]:
# print(out.dtype, expected.dtype, diff.dtype)

In [ ]:
imsave(tmp / "expected.tif", expected[:100])

In [ ]:
imsave(tmp / "out.tif", out[:100].astype("uint16"))


In [ ]:
print(out[400:420].min(), out[400:420].max())


In [ ]:
print((out > 0).sum() / numpy.prod(numpy.array(out.shape)))